# Do some imports

In [3]:
#do not change this order 
import onnx 
import torch 

!pip install --user pandas
!pip install --user scikit-learn
!pip install --user tqdm


import numpy as np
from numpy import genfromtxt
from tqdm import tqdm

from torch.utils.data import DataLoader, Dataset
from brevitas.core.quant import QuantType
import brevitas.onnx as bo
import onnx
from finn.util.visualization import showSrc, showInNetron

from dataloader import UNSW_NB15
from dataloader_quantized import UNSW_NB15_quantized

# Brevitas import and visualization

In [18]:
showInNetron('brevitas_w1_a1NSW_NB15_model.onnx')


Stopping http://0.0.0.0:8081
Serving 'brevitas_w1_a1NSW_NB15_model.onnx' at http://0.0.0.0:8081


# Get the model and prepare it

Now that we have the model in .onnx format, we can work with it using FINN. For that FINN ModelWrapper is used. It is a wrapper around the ONNX model which provides several helper functions to make it easier to work with the model.

In [19]:
from finn.core.modelwrapper import ModelWrapper
model_for_sim = ModelWrapper("brevitas_w1_a1NSW_NB15_model.onnx")

In [20]:
from finn.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.infer_shapes import InferShapes
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.fold_constants import FoldConstants

model_for_sim = model_for_sim.transform(InferShapes())
model_for_sim = model_for_sim.transform(FoldConstants())
model_for_sim = model_for_sim.transform(GiveUniqueNodeNames())
model_for_sim = model_for_sim.transform(GiveReadableTensorNames())
model_for_sim = model_for_sim.transform(InferDataTypes())
model_for_sim = model_for_sim.transform(RemoveStaticGraphInputs())

# Simulate the model node by node and get all outputs

This execution function and onnxruntime is used when `execute_onnx` from `onnx_exec` is applied to the model. The model is then simulated node by node and the result is stored in a context dictionary, which contains the values of each tensor at the end of the execution. To get the result, only the output tensor has to be extracted.

The procedure is shown below. We take the model right before the nodes should be converted into HLS layers and generate an input tensor to pass to the execution function. The input tensor is generated from the Brevitas example inputs.

In [6]:
import finn.core.onnx_exec as oxe

In [7]:
test_quantized_dataset = UNSW_NB15_quantized(file_path_train='data/UNSW_NB15_training-set.csv', \
                                              file_path_test = "data/UNSW_NB15_testing-set.csv", \
                                              train=False)
input_tensor = test_quantized_dataset.data[:,:-1]
input_tensor.shape

torch.Size([82332, 594])


torch.Size([82332, 593])

In [21]:
prev_i = 0
for i in range(2287, 82333, 2287): 
    np_array_k_rows = input_tensor[prev_i:i].detach().numpy() # select only the first k rows
    
    input_dict = {"global_in": np_array_k_rows} # create the dictionary
    
    output_dict = oxe.execute_onnx(model_for_sim, input_dict) #execute each node
    output_pysim = output_dict[list(output_dict.keys())[0]] #get the output

    finn_partial_array_before_sigmoid = output_pysim    
    if prev_i == 0:
        finn_array_before_sigmoid = finn_partial_array_before_sigmoid
    else:
        finn_array_before_sigmoid = np.append(finn_array_before_sigmoid, finn_partial_array_before_sigmoid, axis=0)

    prev_i = i
    if i > 10_000:
        break

# Compare with Brevitas

In [22]:
brevitas_array_before_sigmoid = genfromtxt('brevitas_1_bit_model_no_sigmoid.csv', delimiter=',').reshape(-1,1)
np.isclose(finn_array_before_sigmoid, brevitas_array_before_sigmoid[:11435], atol=1e-1).sum()

11435